In [5]:
from azureml.core import Workspace
from azureml.core import ScriptRunConfig
from azureml.core import Environment
from azureml.core import Experiment
from azureml.core import Datastore, Dataset

ws = Workspace.from_config()
compute_target = ws.compute_targets['sla-k80']
PROJECT_ROOT = '../../'
# environment = Environment.from_pip_requirements('env', PROJECT_ROOT+'../requirements_azure')
ds = ws.get_default_datastore()

In [6]:
# data_ref = ds.path('preprocessed.zip').as_download()
# print(data_ref.to_config().__dict__)

input_data = Dataset.File.from_files(ds.path('0.25-0.50/')).as_named_input('input').as_mount()
print(input_data)

In [7]:
# data_ref = ds.path('preprocessed.zip').as_download()
# path_on_compute='~/cloudfiles/code/Users/sascha.lange/data/'
# command = f'bash setup.sh && python baselines/supervised_learning/modelling/train.py --t {str(data_ref)}'.split()
# command = f'bash setup.sh && python baselines/supervised_learning/modelling/train.py'.split()
command = ['bash',
 'setup.sh',
 '&&',
 'python',
 'baselines/supervised_learning/modelling/train.py', '-t', input_data]
config = ScriptRunConfig(
    command=command,
    compute_target=compute_target,  # compute target used to run train.py script
    source_directory=PROJECT_ROOT
    # environment=environment,
    # script='baselines/supervised_learning/modelling/train.py',
)

# config.run_config.data_references[data_ref.data_reference_name] = data_ref.to_config()

In [8]:
exp = Experiment(ws, 'supervised_baseline_training')
exp.start_logging()
run = exp.submit(config)
print(run)
run.wait_for_completion(show_output=True)

Run(Experiment: supervised_baseline_training,
Id: supervised_baseline_training_1647987601_670c7cb8,
Type: azureml.scriptrun,
Status: Preparing)
RunId: supervised_baseline_training_1647987601_670c7cb8
Web View: https://ml.azure.com/runs/supervised_baseline_training_1647987601_670c7cb8?wsid=/subscriptions/2eab4702-c402-4a9c-8fa9-a432d38f3551/resourcegroups/master-sla/workspaces/master-sla-ws&tid=291d05b0-1020-4f36-aac5-b26d20661fe5

Streaming azureml-logs/55_azureml-execution-tvmps_9303dbd08d779d91f2f2915cff84c89b5e3035ae6043bf94e736b027e68ff53f_d.txt

2022-03-22T22:20:40Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/master-sla-ws/azureml/supervised_baseline_training_1647987601_670c7cb8/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/master-sla-ws/azureml/supervised_baseline_training_1647987601_670c7cb8/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=256987 -o nonempty -o allow

ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}